In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = psycopg2.connect(
                        database="postgres", 
                        user='postgres',
                        password='123', 
                        host='localhost', 
                        port='5432'
                    )
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [3]:
import psycopg2 as pg_driver

db = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='123', 
                        host='localhost', 
                        port='5432'
                    )



def execute_queries(db, sql_commands):
    db.autocommit = True
    with db.cursor() as cursor:
        for sql_command in sql_commands:
            print(sql_command)
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                


In [4]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='123', 
                        host='localhost', 
                        port='5432'
                    );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [5]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark'),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Dima'),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Phill'),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Viktor'),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Ivan')
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 3),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 2),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 1)
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (1, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5)
                 

                 
            
        """

execute_query(query)

query = """select l.user_id, u.name 
           from likes l
           left join users u
           on l.user_id=u.user_id
           where l.user_id=1 
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Mark')
1 Record(user_id=1, name='Mark')
2 Record(user_id=1, name='Mark')


In [6]:
query = """select
                l.user_id as from_id,
                u.name as from_name,
                l.user_id_give as to_id,
                us.name as to_name
           from likes l
           left join users u
           on l.user_id=u.user_id
           left join users us
           on l.user_id_give=us.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_id=5, from_name='Ivan', to_id=1, to_name='Mark')
1 Record(from_id=4, from_name='Viktor', to_id=2, to_name='Dima')
2 Record(from_id=3, from_name='Phill', to_id=3, to_name='Phill')
3 Record(from_id=2, from_name='Dima', to_id=4, to_name='Viktor')
4 Record(from_id=1, from_name='Mark', to_id=4, to_name='Viktor')
5 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')
6 Record(from_id=2, from_name='Dima', to_id=5, to_name='Ivan')
7 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')


In [7]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark')
 
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4)
            
        """

execute_query(query)

query = """select
                --l.user_id as from_id,
                u.name as from_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_name='Viktor', like_count=1)
1 Record(from_name='Phill', like_count=1)
2 Record(from_name='Ivan', like_count=1)
3 Record(from_name='Mark', like_count=4)
4 Record(from_name='Dima', like_count=2)


In [8]:
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(to_id=4, to_name='Viktor', like_count=3)
1 Record(to_id=1, to_name='Mark', like_count=1)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=5, to_name='Ivan', like_count=3)


In [9]:
query = """select t1.user_id, t1.user_id_give,
           count(*) as like_count
                
           from likes t1
           inner join likes t2
           on t1.user_id_give=t2.user_id
           where t2.user_id_give=t1.user_id 
           group by t1.user_id_give, t1.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=5, user_id_give=1, like_count=2)
1 Record(user_id=4, user_id_give=2, like_count=1)
2 Record(user_id=3, user_id_give=3, like_count=1)
3 Record(user_id=2, user_id_give=4, like_count=1)
4 Record(user_id=1, user_id_give=5, like_count=2)


In [10]:
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           order by l.user_id_give DESC
           limit 5
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value) 

0 Record(to_id=5, to_name='Ivan', like_count=3)
1 Record(to_id=4, to_name='Viktor', like_count=3)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=1, to_name='Mark', like_count=1)


In [11]:
#Задание 2
import psycopg2 as pg_driver

db = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='123', 
                        host='localhost', 
                        port='5432'
                    )



def execute_queries(db, sql_commands):
    db.autocommit = True
    with db.cursor() as cursor:
        for sql_command in sql_commands:
            print(sql_command)
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS photos;",
                "DROP TABLE IF EXISTS comments;"
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL
                );
                """,
                """CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         photo_name    TEXT       NOT NULL,
                         user_id    INT       NOT NULL
                  );
                """,
               """CREATE TABLE comments (
                         comment_id    INT       NOT NULL,
                         author_id    INT       NOT NULL,
                         comment_text    TEXT       NOT NULL,
                         user_id    INT       NOT NULL,
                         photo_id    INT       NOT NULL
                );
                """,
               """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         type       TEXT       NOT NULL,
                         type_id    INT       NOT NULL,
                         to_user_id     INT       NOT NULL    
                         
                );
                """]
                

execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS photos;
DROP TABLE IF EXISTS comments;DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL
                );
                
CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         photo_name    TEXT       NOT NULL,
                         user_id    INT       NOT NULL
                  );
                
CREATE TABLE comments (
                         comment_id    INT       NOT NULL,
                         author_id    INT       NOT NULL,
                         comment_text    TEXT       NOT NULL,
                         user_id    INT       NOT NULL,
                         photo_id    INT       NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         type       TEXT       NOT NULL,
  

In [12]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = psycopg2.connect(
                                database="postgres", 
                                user='postgres',
                                password='123', 
                                host='localhost', 
                                port='5432'
                     )
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [13]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='123', 
                        host='localhost', 
                        port='5432'
                    );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [14]:
query = """ INSERT INTO users (user_id, name) 
            VALUES 
                 (1, 'Elena'),
                 (2, 'Dina'),
                 (3, 'Mona'),
                 (4, 'Anastasia'),
                 (5, 'Bob'),
                 (6, 'Lisa'),
                 (7, 'Jake')
                 
            
        """

execute_query(query)

query = """ INSERT INTO photos (photo_id, photo_name, user_id ) 
            VALUES 
                 (1, 'Waterfall', 4),
                 (2, 'Sea', 7),
                 (3, 'Dog', 3),
                 (4, 'Breakfast', 7),
                 (5, 'Mother', 2),
                 (6, 'Love', 1),
                 (7, 'Home', 4)
                 
        """    

execute_query(query)

query = """ INSERT INTO comments (comment_id, author_id, comment_text, user_id, photo_id ) 
            VALUES 
                 (1, 2, 'WOW', 7, 2),
                 (2, 5, 'Delicious', 7, 4),
                 (3, 3, 'Miss you', 1, 6),
                 (4, 6, 'Cool', 4, 1)
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, type, type_id, to_user_id) 
            VALUES 
                 (1, 'photo', 4, 7),
                 (2, 'user', 6, 6),
                 (3, 'comment', 2, 5),
                 (4, 'user', 1, 1),
                 (5, 'photo', 6, 1)
            
        """

execute_query(query)

In [15]:
 all_rows = execute_query("select * from users", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Elena')
1 Record(user_id=2, name='Dina')
2 Record(user_id=3, name='Mona')
3 Record(user_id=4, name='Anastasia')
4 Record(user_id=5, name='Bob')
5 Record(user_id=6, name='Lisa')
6 Record(user_id=7, name='Jake')


In [16]:
query = """select
                u.name,
                --p.photo_name,
                count(*) as photo_count
                
           from users u
           left join photos p
           on u.user_id=p.user_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(name='Anastasia', photo_count=2)
1 Record(name='Lisa', photo_count=1)
2 Record(name='Dina', photo_count=1)
3 Record(name='Jake', photo_count=2)
4 Record(name='Mona', photo_count=1)
5 Record(name='Bob', photo_count=1)
6 Record(name='Elena', photo_count=1)


In [17]:
query = """select
                l.user_id,
                u.name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by l.user_id, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=3, name='Mona', like_count=1)
1 Record(user_id=4, name='Anastasia', like_count=1)
2 Record(user_id=5, name='Bob', like_count=1)
3 Record(user_id=2, name='Dina', like_count=1)
4 Record(user_id=1, name='Elena', like_count=1)


In [18]:
query = """select
                l.to_user_id,
                u.name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.to_user_id=u.user_id
           group by l.to_user_id, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(to_user_id=6, name='Lisa', like_count=1)
1 Record(to_user_id=5, name='Bob', like_count=1)
2 Record(to_user_id=7, name='Jake', like_count=1)
3 Record(to_user_id=1, name='Elena', like_count=2)
